# Projet 2 - Fashion Trend Intelligence | Segmentation vestimentaire avec IA

Ce notebook permet d’évaluer la faisabilité technique du modèle SegFormer-clothes, afin de déterminer s’il est capable d’identifier et d’isoler avec précision chaque pièce vestimentaire présente dans une image.


## 1. Installation du projet et de son environnement

Afin d’utiliser correctement ce notebook, vérifiez que vous disposez du bon environnement pour l’exécuter.


### 1.1 Installation de Python

Pour ce projet, il est nécessaire d’avoir **au minimum Python 3.8**.  
Si ce n’est pas déjà le cas, vous pouvez vous référer à [la documentation officielle](https://www.python.org/downloads/).

Vérifiez votre version de Python :

```bash
python --version
```

### 1.2 Installation de `uv`

`uv` est un gestionnaire de projets Python permettant d’installer et d’organiser les dépendances plus rapidement et plus simplement que les outils traditionnels (`pip`, `virtualenv`, etc.).

Pour installer `uv`, veuillez suivre [la documentation officielle](https://docs.astral.sh/uv/getting-started/installation/#standalone-installer)

Vérifiez l’installation :

```bash
uv --version
```

### 1.3 Création du projet

Créez un nouveau projet Python avec `uv` :

```bash
uv init nom_du_projet
cd nom_du_projet
```

La structure de base du projet est alors générée automatiquement.


### 1.4 Création et activation de l’environnement virtuel

Créez l’environnement virtuel :

```bash
uv venv
```

Activez-le selon votre système :

* **Linux / macOS**

```bash
source .venv/bin/activate
```

* **Windows (PowerShell)**

```powershell
.venv\Scripts\Activate.ps1
```

Voir la documentation officielle :
[https://docs.astral.sh/uv/pip/environments/#creating-a-virtual-environment](https://docs.astral.sh/uv/pip/environments/#creating-a-virtual-environment)



### 1.5 Installation des dépendances

> ❗ Assurez-vous que l’environnement virtuel est **activé** avant d’installer les dépendances.

Installez les bibliothèques nécessaires au projet :

```bash
uv add ipykernel jupyterlab requests pillow matplotlib numpy tqdm python-dotenv aiohttp
```

Ces dépendances sont automatiquement enregistrées dans le fichier `pyproject.toml`.

Vous pouvez visualiser l’ensemble des dépendances installées avec la commande suivante :

```bash
uv tree
```

### 1.6 Création du fichier `.env`

Créez un fichier nommé `.env` à la racine du projet.

Copiez-y le contenu suivant et remplacez les valeurs si nécessaire :

```env
# Token d'authentification à l'API Hugging Face
HF_API_TOKEN=VOTRE_TOKEN_HUGGING_FACE_ICI

# Chemins du dataset
DATASET_IMAGES_DIR=content/images_a_segmenter
DATASET_ANNOTATIONS_DIR=content/annotations
```

⚠️ **Important** :

* Ne partagez jamais votre fichier `.env`
* Ajoutez-le à votre `.gitignore`

```gitignore
.env
```

### 1.7 Création d’un token Hugging Face

1. Créez un compte sur [https://huggingface.co/](https://huggingface.co/)
2. Allez dans **Profile → Settings → Access Tokens**
3. Créez un nouveau token (rôle **read** suffisant)
4. Copiez le token dans la variable `HF_API_TOKEN` du fichier `.env`


### 1.8 Préparation des jeux de données

Récupérez les jeux de données (images et annotations) et placez-les dans les dossiers suivants :

```
content/
├── images_a_segmenter/
│   ├── image_0.jpg
│   ├── image_1.jpg
│   └── ...
└── annotations/
    ├── mask_0.json
    ├── mask_1.json
    └── ...
```

## 2. Importation des Bibliothèques

Commençons par importer les bibliothèques Python nécessaires. Nous aurons besoin de :
- **`os`** : Interaction avec le système de fichiers (navigation dans les répertoires, listage des fichiers images).
- **`requests`** : Envoi de requêtes HTTP vers l'API pour la segmentation d'images.
- **`PIL (Pillow)`** : Manipulation et traitement des images (ouverture, redimensionnement, conversion).
- **`matplotlib.pyplot`** : Visualisation des images originales et des masques de segmentation.
- **`matplotlib.patches`** : Création d'éléments graphiques personnalisés pour les légendes des visualisations.
- **`numpy`** : Manipulation efficace des tableaux numériques représentant les pixels des images.
- **`tqdm.notebook`** : Affichage d'une barre de progression interactive dans les notebooks Jupyter (utile lors du traitement par lot).
- **`base64`** : Encodage/décodage en Base64 des images et masques échangés avec l'API.
- **`io`** : Gestion des flux de données en mémoire pour la manipulation des images sans fichiers temporaires.
- **`python-dotenv`** : Chargement sécurisé des variables d'environnement (comme les clés API) depuis un fichier `.env`.
- **`time`** : Gestion des délais entre les appels API pour respecter les limites de taux (rate limiting).
- **`re`** : Traitement d'expressions régulières (parsing de réponses, validation de formats).
- **`aiohttp`** : Requêtes HTTP asynchrones pour améliorer les performances lors du traitement par lot d'images.


In [16]:
import os
import requests
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import numpy as np
from tqdm.notebook import tqdm
from tqdm.asyncio import tqdm as async_tqdm
import base64
import io
from dotenv import load_dotenv
import time
import re
import asyncio
import aiohttp
from typing import Dict, List
from datetime import datetime


## 3. Chargement de la Configuration

In [17]:
# Charger les variables d'environnement depuis le fichier .env
load_dotenv(override=True)

# Récupération des variables d'environnement
IMAGES_DIR = os.getenv('DATASET_IMAGES_DIR')
ANNOTATIONS_DIR = os.getenv('DATASET_ANNOTATIONS_DIR')
API_TOKEN = os.getenv('HF_API_TOKEN')
MAX_IMAGES = 3  # Nombre maximum d'images à traiter

# Vérification de la présence des variables
if not IMAGES_DIR or not API_TOKEN or not ANNOTATIONS_DIR:
    raise ValueError("Variables d'environnement manquantes dans le fichier .env")

# Vérification du token
if API_TOKEN == "VOTRE_TOKEN_HUGGING_FACE_ICI":
    raise ValueError("Vous devez remplacer 'VOTRE_TOKEN_HUGGING_FACE_ICI' par votre token API personnel dans le fichier .env")

# Création des dossiers s'ils n'existent pas
for directory, name in [(IMAGES_DIR, "images"), (ANNOTATIONS_DIR, "annotations")]:
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"✓ Dossier '{directory}' créé pour les {name}.")
    else:
        print(f"✓ Dossier '{directory}' existant.")

✓ Dossier 'content/images_a_segmenter' existant.
✓ Dossier 'content/annotations' existant.


## 4. Configuration de l'API Hugging Face

### 4.1 Présentation du modèle
Nous utilisons le modèle [**SegFormer-B3-Clothes**](https://huggingface.co/sayeed99/segformer_b3_clothes) qui est spécialisé dans la segmentation sémantique de vêtements. Ce modèle peut détecter 18 classes différentes :
- Éléments vestimentaires : chapeau, haut, jupe, pantalon, robe, ceinture, chaussures, sac, écharpe
- Parties du corps : cheveux, visage, bras, jambes
- Accessoires : lunettes de soleil
- Arrière-plan

### 4.2 Configuration de l'endpoint API
L'API Hugging Face Inference Router permet d'interroger le modèle sans avoir à le déployer localement. Voici les éléments de configuration :

**URL de l'API** : `https://router.huggingface.co/hf-inference/models/sayeed99/segformer_b3_clothes`

**Headers requis** :
- `Authorization: Bearer YOUR_TOKEN` : Authentification avec votre token personnel
- `Content-Type: image/jpeg` ou `image/png` : Type MIME de l'image envoyée

**Format de réponse** : L'API retourne un JSON contenant une liste de masques encodés en base64, un par classe détectée.

**Sécurité** : Ne jamais hardcoder votre token dans le notebook. Elle doit mis dans la variable d'environnement `HF_API_TOKEN'` le fichier `.env`

### 4.3 Bonnes pratiques d'utilisation
1. **Rate Limiting** : Espacer les requêtes de 1-2 secondes pour éviter les erreurs 429 (Too Many Requests)
2. **Timeout** : Définir un timeout de 30 secondes minimum (le modèle peut être lent)
3. **Gestion d'erreurs** : Prévoir les cas suivants :
   - 401 : Token invalide
   - 429 : Trop de requêtes
   - 500 : Erreur serveur (réessayer après quelques secondes)
   - 503 : Modèle en cours de chargement (attendre et réessayer)

In [ ]:
API_URL = "https://router.huggingface.co/hf-inference/models/sayeed99/segformer_b3_clothes"
headers = {
    "Authorization": f"Bearer {API_TOKEN}"
    # Le "Content-Type" sera ajouté dynamiquement lors de l'envoi de l'image
}


# Rate limiting pour compte gratuit (d'après la doc HF)
# Inference API : ~1000 requêtes/jour, avec burst de ~100 requêtes/min
REQUESTS_PER_MINUTE = 30  # On reste prudent (bien en dessous de 100)
DELAY_BETWEEN_REQUESTS = 60 / REQUESTS_PER_MINUTE  # ~2 secondes entre chaque requête


# Configuration des requêtes asynchrones
MAX_CONCURRENT_REQUESTS = 5      # Nombre de requêtes simultanées maximum
MAX_RETRIES = 3                  # Nombre de tentatives en cas d'erreur
INITIAL_RETRY_DELAY = 2          # Délai initial entre les tentatives (secondes)
REQUEST_TIMEOUT = 30             # Timeout des requêtes (secondes)

async def segment_clothes_async(
    session: aiohttp.ClientSession,
    single_image_path: str,
    semaphore: asyncio.Semaphore
) -> Dict:
    """
    Requête asynchrone à l'API Hugging Face avec rate limiting.

    Args:
        session: Session aiohttp partagée
        single_image_path: Chemin vers l'image à segmenter
        semaphore: Sémaphore pour limiter les requêtes concurrentes

    Returns:
        dict: Résultats de la segmentation (JSON de l'API)
        
    Raises:
        FileNotFoundError: Si le fichier image n'existe pas
        ValueError: Si le format d'image n'est pas supporté
        aiohttp.ClientError: Si la requête API échoue
    """
    
    # Vérification de l'existence du fichier
    if not os.path.exists(single_image_path):
        raise FileNotFoundError(f"L'image '{single_image_path}' n'existe pas")
    
    # Vérification du format d'image
    try:
        with Image.open(single_image_path) as img:
            image_format = img.format
            
            if image_format not in ['JPEG', 'PNG']:
                raise ValueError(
                    f"Format '{image_format}' non supporté. "
                    f"Formats acceptés: JPEG, PNG"
                )
    except Exception as e:
        raise ValueError(f"Impossible d'ouvrir l'image: {e}")
    
    # Lecture du contenu binaire
    try:
        with open(single_image_path, 'rb') as fichier:
            image_data = fichier.read()
    except IOError as e:
        raise IOError(f"Erreur lors de la lecture du fichier: {e}")
    
    # Configuration des headers
    request_headers = headers.copy()
    request_headers["Content-Type"] = f"image/{image_format.lower()}"
    
    # Utilisation du sémaphore pour limiter les requêtes concurrentes
    async with semaphore:
        retry_delay = INITIAL_RETRY_DELAY
        
        for attempt in range(MAX_RETRIES):
            try:
                async with session.post(
                    API_URL,
                    headers=request_headers,
                    data=image_data,
                    timeout=aiohttp.ClientTimeout(total=REQUEST_TIMEOUT)
                ) as response:
                    
                    # Gestion des différents codes de statut
                    if response.status == 503:
                        # Modèle en cours de chargement
                        print(f"⏳ Modèle en chargement, attente {INITIAL_RETRY_DELAY}s...")
                        await asyncio.sleep(INITIAL_RETRY_DELAY)
                        retry_delay *= 2  # Backoff exponentiel
                        continue
                    
                    elif response.status == 429:
                        # Rate limit atteint
                        retry_after = response.headers.get('Retry-After', retry_delay)
                        wait_time = int(retry_after) if isinstance(retry_after, str) and retry_after.isdigit() else retry_delay
                        print(f"⚠️ Rate limit atteint, attente {wait_time}s...")
                        await asyncio.sleep(wait_time)
                        continue
                    
                    elif response.status == 401:
                        raise ValueError("Token API invalide ou expiré")
                    
                    elif response.status >= 500:
                        # Erreur serveur, on réessaie
                        if attempt < MAX_RETRIES - 1:
                            print(f"❌ Erreur serveur (5xx), nouvelle tentative dans {retry_delay}s...")
                            await asyncio.sleep(retry_delay)
                            retry_delay *= 2
                            continue
                        else:
                            raise aiohttp.ClientError(
                                f"Erreur serveur persistante (status {response.status})"
                            )
                    
                    # Vérification du statut
                    response.raise_for_status()
                    
                    # Parsing de la réponse
                    results = await response.json()
                    print(f"✅ Réponse reçue pour {os.path.basename(single_image_path)}")
                    
                    # Délai entre les requêtes pour respecter le rate limit
                    await asyncio.sleep(DELAY_BETWEEN_REQUESTS)
                    
                    return results
                    
            except asyncio.TimeoutError:
                if attempt < MAX_RETRIES - 1:
                    print(f"⏱️ Timeout, nouvelle tentative...")
                    await asyncio.sleep(retry_delay)
                    continue
                else:
                    raise asyncio.TimeoutError(
                        f"Timeout après {MAX_RETRIES} tentatives pour '{single_image_path}'"
                    )
            
            except aiohttp.ClientError as e:
                if attempt < MAX_RETRIES - 1:
                    print(f"❌ Erreur réseau, nouvelle tentative dans {retry_delay}s...")
                    await asyncio.sleep(retry_delay)
                    continue
                else:
                    raise
        
        raise Exception(f"Échec après {MAX_RETRIES} tentatives")


async def segment_clothes_batch(image_paths: List[str]) -> List[Dict]:
    """
    Segmente un lot d'images de manière asynchrone avec rate limiting.
    
    Args:
        image_paths: Liste des chemins d'images à traiter
        
    Returns:
        List[Dict]: Liste des résultats de segmentation
    """
    # Sémaphore pour limiter les requêtes concurrentes
    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
    
    async with aiohttp.ClientSession() as session:
        tasks = [
            segment_clothes_async(session, img_path, semaphore)
            for img_path in image_paths
        ]
        
        # Exécution de toutes les tâches
        results = await async_tqdm.gather(*tasks, desc="Segmentation")        
        
        # Traitement des résultats et des erreurs
        processed_results = []
        for i, result in enumerate(results):
            if isinstance(result, Exception):
                print(f"❌ Erreur pour {image_paths[i]}: {result}")
                processed_results.append(None)
            else:
                processed_results.append(result)
        
        return processed_results


# Fonction wrapper pour utiliser dans un notebook Jupyter
def segment_images(image_paths: List[str]) -> List[Dict]:
    """
    Fonction synchrone wrapper pour utilisation facile dans le notebook.
    
    Args:
        image_paths: Liste des chemins d'images (ou chemin unique)
        
    Returns:
        List[Dict] ou Dict: Résultats de la segmentation
    """
    # Accepter un seul chemin ou une liste
    if isinstance(image_paths, str):
        image_paths = [image_paths]
    
    print(f"🚀 Début de la segmentation de {len(image_paths)} image(s)...")
    print(f"⏱️ Délai entre requêtes: {DELAY_BETWEEN_REQUESTS:.2f}s")
    
    start_time = datetime.now()
    
    # Exécution asynchrone
    results = asyncio.run(segment_clothes_batch(image_paths))
    
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()
    
    print(f"\n✅ Traitement terminé en {duration:.2f}s")
    print(f"📊 Succès: {sum(1 for r in results if r is not None)}/{len(image_paths)}")
    print(f"⚡ Vitesse moyenne: {len(image_paths)/duration:.2f} images/seconde")

    # Si une seule image, retourner directement le résultat
    if len(image_paths) == 1:
        return results[0]
    
    return results

## 5. Fonctions Utilitaires pour le Traitement des Masques

Le modèle que nous utilisons (`sayeed99/segformer_b3_clothes`) renvoie des masques pour différentes classes (cheveux, chapeau, etc.). Ces masques sont encodés en base64. Les fonctions ci-dessous sont fournies pour vous aider à :
1.  `CLASS_MAPPING`: Un dictionnaire qui associe les noms de classes (ex: "Hat") à des identifiants numériques.
2.  `get_image_dimensions`: Récupérer les dimensions d'une image.
3.  `decode_base64_mask`: Décoder un masque de base64 en une image (tableau NumPy) et le redimensionner.
4.  `create_masks`: Combiner les masques de toutes les classes détectées en un seul masque de segmentation final, où chaque pixel a la valeur de l'ID de sa classe.

In [ ]:
CLASS_MAPPING = {
    "Background": 0,
    "Hat": 1,
    "Hair": 2,
    "Sunglasses": 3,
    "Upper-clothes": 4,
    "Skirt": 5,
    "Pants": 6,
    "Dress": 7,
    "Belt": 8,
    "Left-shoe": 9,
    "Right-shoe": 10,
    "Face": 11,
    "Left-leg": 12,
    "Right-leg": 13,
    "Left-arm": 14,
    "Right-arm": 15,
    "Bag": 16,
    "Scarf": 17
}

def get_image_dimensions(img_path):
    """
    Get the dimensions of an image.

    Args:
        img_path (str): Path to the image.

    Returns:
        tuple: (width, height) of the image.
    """
    original_image = Image.open(img_path)
    return original_image.size

def decode_base64_mask(base64_string, width, height):
    """
    Decode a base64-encoded mask into a NumPy array.

    Args:
        base64_string (str): Base64-encoded mask.
        width (int): Target width.
        height (int): Target height.

    Returns:
        np.ndarray: Single-channel mask array.
    """
    mask_data = base64.b64decode(base64_string)
    mask_image = Image.open(io.BytesIO(mask_data))
    mask_array = np.array(mask_image)
    if len(mask_array.shape) == 3:
        mask_array = mask_array[:, :, 0]  # Take first channel if RGB
    mask_image = Image.fromarray(mask_array).resize((width, height), Image.NEAREST)
    return np.array(mask_image)

def create_masks(results, width, height):
    """
    Combine multiple class masks into a single segmentation mask.

    Args:
        results (list): List of dictionaries with 'label' and 'mask' keys.
        width (int): Target width.
        height (int): Target height.

    Returns:
        np.ndarray: Combined segmentation mask with class indices.
    """
    combined_mask = np.zeros((height, width), dtype=np.uint8)  # Initialize with Background (0)

    # Process non-Background masks first
    for result in results:
        label = result['label']
        class_id = CLASS_MAPPING.get(label, 0)
        if class_id == 0:  # Skip Background
            continue
        mask_array = decode_base64_mask(result['mask'], width, height)
        combined_mask[mask_array > 0] = class_id

    # Process Background last to ensure it doesn't overwrite other classes unnecessarily
    # (Though the model usually provides non-overlapping masks for distinct classes other than background)
    for result in results:
        if result['label'] == 'Background':
            mask_array = decode_base64_mask(result['mask'], width, height)
            # Apply background only where no other class has been assigned yet
            # This logic might need adjustment based on how the model defines 'Background'
            # For this model, it seems safer to just let non-background overwrite it first.
            # A simple application like this should be fine: if Background mask says pixel is BG, set it to 0.
            # However, a more robust way might be to only set to background if combined_mask is still 0 (initial value)
            combined_mask[mask_array > 0] = 0 # Class ID for Background is 0

    return combined_mask